In [329]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, 
)

# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc
# from nlpaug.util import Action

# from googletrans import Translator
# import translators as ts

import re, math, random, json
from copy import deepcopy
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from tqdm import tqdm

from module.preprocess import preprocess

# import demoji

# from cleantext import clean
# from pykospacing import Spacing
# from hanspell import spell_checker

In [330]:
# train_json = './dataset//nikluge-sa-2022-train.jsonl'
# train_raw = pd.read_json(train_json, lines=True)
# train_raw.annotation = train_raw.annotation.apply(lambda x: str(x))
# train_raw = train_raw[['sentence_form', 'annotation']]
# dup = train_raw.duplicated(keep=False)
# train_raw[dup]

In [331]:
# dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
# dev_raw = pd.read_json(dev_json, lines=True)
# dev_raw.annotation = dev_raw.annotation.apply(lambda x: str(x))
# dev_raw = dev_raw[['sentence_form', 'annotation']]
# dup = dev_raw.duplicated(keep=False)
# dev_raw[dup]

In [332]:
train_json = './dataset/nikluge-sa-2022-train.jsonl'
dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
test_json = './dataset/nikluge-sa-2022-test.jsonl'

In [333]:
train = pd.read_json(train_json, lines=True)
dev = pd.read_json(dev_json, lines=True)
test = pd.read_json(test_json, lines=True)

In [334]:
# train.iloc[2319]
train = train.drop(2319)
# dev.iloc[1692]
dev = dev.drop(1692)

# Preprocess

## Cleansing

### Before

In [335]:
for el in train.sample(n=5).sentence_form:
    print(el)

트러블 이제 안뇽~~
2중안감에 내장몰드로 티나지않고요.
끈적이지 않은 제형으로 아침저녁으로 세안후 바르기 딱좋아용
크림쓰고 엄청 만족스러웠는데 치약도 맘에 쏙! ㅋ
볼륨쿠션 리얼 엠보싱 원단이라 잘닦이고 독일 피부과학 연구소 더마 테스트 인증까지


In [336]:
train.sentence_form = train.sentence_form.apply(preprocess)
dev.sentence_form = dev.sentence_form.apply(preprocess)
test.sentence_form = test.sentence_form.apply(preprocess)
total = pd.concat([train, dev])

Test

In [337]:
case = total.sentence_form.str.contains('r[^A-Za-z0-9가-힣\s]+', case=False, flags=0, na=None, regex=True)
for e in total[case].sentence_form:
    print(e)

### After

In [338]:
for i, row in total[['id', 'sentence_form']].sample(n=5).iterrows():
    print(row.id, '\t', row.sentence_form)

nikluge-sa-2022-dev-00439 	 저자극성 커버는 항균 처리가 되어 있어서 굉장히 상쾌함을 더해주고요
nikluge-sa-2022-train-01282 	 피부에 밀리지 않고 보송보송한 피부표현도 개인적으로 너무 좋았어요
nikluge-sa-2022-train-02958 	 픽앤웨어 욜라 꼼꼼해
nikluge-sa-2022-train-00838 	 왜들 좋아하는지 이유를 알겠슴
nikluge-sa-2022-train-00856 	 하트라인을 크게 좋아하진 않는데 요곤 맘에 든다


In [339]:
# total['check'] = total.sentence_form.str.find('OO')
# for row in total[total.check > -1].sentence_form:
#     print(row)
#     break

In [340]:
total

,id,sentence_form,annotation
0,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,"[[본품#품질, [기어, 16, 18], negative]]"
1,nikluge-sa-2022-train-00002,이거 뭐 삐꾸를 준 거 아냐 불안하고 거금 투자한 게 왜 이래 싶어서 정이 확 떨어...,"[[본품#품질, [기어 텐션, 67, 72], negative]]"
2,nikluge-sa-2022-train-00003,간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다,"[[제품 전체#일반, [None, 0, 0], positive]]"
3,nikluge-sa-2022-train-00004,샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이...,"[[제품 전체#일반, [샥이 없는 모델, 0, 8], neutral]]"
4,nikluge-sa-2022-train-00005,안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다,"[[본품#일반, [안장, 0, 2], negative]]"
...,...,...,...
2789,nikluge-sa-2022-dev-02790,썸머세트라고 해서 세럼 선크림 이렇게 패키지로 파는 것 괜찮음,"[[패키지/구성품#일반, [썸머세트, 0, 4], positive]]"
2790,nikluge-sa-2022-dev-02791,비싸다,"[[제품 전체#가격, [None, 0, 0], negative]]"
2791,nikluge-sa-2022-dev-02792,비싸지만 건강이 더 비싸니까,"[[제품 전체#가격, [None, 0, 0], negative]]"
2792,nikluge-sa-2022-dev-02793,대형으로 샀더니 잘 맞음,"[[제품 전체#일반, [대형, 0, 2], positive]]"


## Reformat

In [341]:
### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'
]

more_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

In [342]:
len(entity_property_pair)

25

In [343]:
def reformat(df):
    ep =[]
    p = []
    for index, row in df.iterrows():
        utterance = row.sentence_form
        id = row.id

        for pair in entity_property_pair:
            isPairInOpinion = False
            if pd.isna(utterance):
                break
            for annotation in row.annotation:
                entity_property = annotation[0]
                sentiment = annotation[2]
                if entity_property not in entity_property_pair:
                    print(row.id, entity_property)
                    break
                if sentiment not in polarity_id_to_name:
                    print(row.id, entity_property, sentiment)
                    break
                if entity_property == pair:
                    ep.append([id, utterance, entity_property, tf_name_to_id['True']])
                    p.append([id, utterance, entity_property, sentiment])
                    isPairInOpinion = True
                    break

            if isPairInOpinion is False:
                ep.append([id, utterance, pair, tf_name_to_id['False']])

    return ep, p

In [344]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [345]:
len(train), len(dev)

(3000, 2793)

In [346]:
ep_train, p_train = reformat(train)
ep_train = pd.DataFrame(ep_train, columns=['id', 'sentence_form', 'entity_property', 'labels'])
p_train = pd.DataFrame(p_train, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])

ep_dev, p_dev = reformat(dev)
ep_dev = pd.DataFrame(ep_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])
p_dev = pd.DataFrame(p_dev, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])

len(ep_train), len(ep_dev), len(p_train), len(p_dev)

(75000, 69825, 3196, 3002)

In [347]:
ep_train.sort_values(['id', 'labels'], inplace=True)
ep_dev.sort_values(['id', 'labels'], inplace=True)

In [348]:
ep_total = pd.concat([ep_train, ep_dev])
p_total = pd.concat([p_train, p_dev])
len(ep_total), len(p_total)

(144825, 6198)

In [349]:
ep_total.reset_index(inplace=True, drop=True)
p_total.reset_index(inplace=True, drop=True)

In [350]:
p_total_binary = reformat_p_binary(p_total)
p_total_binary = pd.DataFrame(p_total_binary, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [351]:
p_total_binary.sort_values(['id', 'entity_property', 'labels'], inplace=True, ascending=[True, False, True])

In [352]:
p_dev_binary = reformat_p_binary(p_dev)
p_dev_binary = pd.DataFrame(p_dev_binary, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [353]:
p_dev_binary.sort_values(['id', 'entity_property', 'labels'], inplace=True, ascending=[True, False, True])

In [359]:
test

,id,sentence_form,annotation
0,nikluge-sa-2022-test-00001,하나 사려고 알아보는 중인데 맘에드는거 발견,[]
1,nikluge-sa-2022-test-00002,동양인 피부톤과 잘 어울리고 우아한 분위기를 풍긴다네,[]
2,nikluge-sa-2022-test-00003,근데 이건 마르살라보다 더 지나친 색 같은데,[]
3,nikluge-sa-2022-test-00004,나스 색조가 다 그렇지만서도 어데이셔스 라인은 진짜 색 기막히게 뽑는것 같다,[]
4,nikluge-sa-2022-test-00005,색상만 보면 이걸 어떻게 발라 싶겠지만 의외로 너무너무 괜찮다,[]
...,...,...,...
2122,nikluge-sa-2022-test-02123,간단한 충전으로 간편한 사용이 가능한거죠,[]
2123,nikluge-sa-2022-test-02124,눈을 가린 상태에서도 간편하게 조작이 가능하구요,[]
2124,nikluge-sa-2022-test-02125,다양한 마사지로 관자놀이부터 눈주변까지 부드럽고 강력한 마사지가 진행됩니다,[]
2125,nikluge-sa-2022-test-02126,본체부터 케이블 설명서까지 깔끔하게 정리되어 보관이 가능하니 더더 맘에 쏙 들어요,[]


In [328]:
save_path = './dataset/cleaned_v2_total_v1'
!mkdir -p {save_path}

train.to_csv(f'{save_path}/raw_train.csv', index=False)
dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
test.to_csv(f'{save_path}/raw_test.csv', index=False)

ep_total.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)

p_total_binary.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_dev_binary.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

## Counting

In [ ]:
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))
ep_train = ep_train.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_train = p_train.drop_duplicates()
p_dev = p_dev.drop_duplicates()
p_binary_train = p_binary_train.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
print('\nafter drop_duplicates')
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))

## Save Files

In [ ]:
save_path = './dataset/cleaned_v1'

train.to_csv(f'{save_path}/raw_train.csv', index=False)
dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
test.to_csv(f'{save_path}/raw_test.csv', index=False)

ep_train.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)
p_train.to_csv(f'{save_path}/pc_train.csv', index=False)
p_dev.to_csv(f'{save_path}/pc_dev.csv', index=False)
p_binary_train.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_binary_dev.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

# ASC Augmentation

In [ ]:
model_checkpoint = '/content/drive/MyDrive/aspect_based_sentiment_analysis/base_model/klue_roberta_base/v2/klue_roberta_base_mlm/checkpoint-19860'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
sTokens = tokenizer.all_special_tokens

def delTokens(sent):
    sent = sent.split(' ')
    temp = []
    for e in sent:
        if e not in sTokens:
            temp.append(e)
    return ' '.join(temp)

In [ ]:
positive, negative, neutral = p_train[p_train.sentiment == 'positive'], p_train[p_train.sentiment == 'negative'], p_train[p_train.sentiment == 'neutral']

In [ ]:
len(positive), len(negative), len(neutral)

In [ ]:
(58 * 3) * 4 * 3, (95 * 3) * 4 * 2 # bt ri rr

Back Translation / Random Insertion / Random Replacement / Random Swap / Random Deletion

In [ ]:
def backTrans(text):
    aug1 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='en')
    aug1 = ts.papago(aug1, sleep_seconds=5, from_language='en', to_language='ko')

    aug2 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='ja')
    aug2 = ts.papago(aug2, sleep_seconds=5, from_language='ja', to_language='ko')

    return [aug1, aug2]

def randomInsert(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomReplace(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomSwap(num, sample):
    aug = naw.RandomWordAug(action='swap', aug_min=1, aug_max=1, aug_p=0.3)    
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

def randomSplit(num, sample):
    aug = naw.SplitAug(aug_min=1, aug_max=1, aug_p=0.3, min_char=3)
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

In [ ]:
(58 * 3) * 5 * 4, (95 * 3) * 4 * 3 # bt ri rr

In [ ]:
def backtransRoutine(data2augment, output_path):
    print('back translation started.')
    temp = []
    for row in data2augment:
        augs = backTrans(row[1])
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(len(f'back translation finished.\ncurrent count: {len(data2augment)}'))

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
import os

def edaRoutine(data2augment, ri, rr, output_path):
    print(f'current count: {len(data2augment)}')
    print('random insertion started.')
    temp = []
    for row in data2augment:
        augs = randomInsert(ri, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random insertion finished.\ncurrent count: {len(data2augment)}')

    print('random replacement started.')
    temp = []
    for row in data2augment:
        augs = randomReplace(rr, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random replacement finished.\ncurrent count: {len(data2augment)}')

    print('random swap and split started.')
    while len(data2augment) < len(positive):
        temp = []
        k = random.randrange(len(negative))
        id, text, entity, sentiment = data2augment[k]

        selector = random.randint(0,1)
        if selector == 0:
            augs = randomSwap(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        else:
            augs = randomSplit(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        if len(data2augment)%25 == 0:
            print(f'random swap and split in progress.\ncurrent count: {len(data2augment)}')

    print(f'whole augmentation routine finished.\ntotal count: {len(data2augment)}')

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
### negative
# # back translation

# data2augment = negative.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_bt.csv'
output_path = os.path.join(output_folder, output_file)

# negative_bt = backtransRoutine(data2augment, output_path)
negative_bt = pd.read_csv(output_path)
negative_bt = negative_bt.values.tolist()
# RI / RR

ri = 4 # times - 1
rr = 3 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_aug.csv'
output_path = os.path.join(output_folder, output_file)

# negative_aug = edaRoutine(negative_bt, ri, rr, output_path)
negative_aug = pd.read_csv(output_path)

In [ ]:
negative_aug
negative_aug = negative_aug.drop_duplicates()

In [ ]:
# negative_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# negative_aug.sort_values('id').head(50).sentence_form

In [ ]:
### neutral
# back translation

# data2augment = neutral.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_bt.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_bt = backtransRoutine(data2augment, output_path)
neutral_bt = pd.read_csv(output_path)
neutral_bt = neutral_bt.values.tolist()

# RI / RR

ri = 3 # times - 1
rr = 2 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_aug.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_aug = edaRoutine(neutral_bt, ri, rr, output_path)
neutral_aug = pd.read_csv(output_path)

In [ ]:
neutral_aug
neutral_aug = neutral_aug.drop_duplicates()

In [ ]:
# neutral_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# neutral_aug.sort_values('id').head(50).sentence_form

In [ ]:
p_train_aug = pd.concat([positive, negative_aug, neutral_aug])

In [ ]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [ ]:
p_binary_train_aug = reformat_p_binary(p_train_aug)
p_binary_train_aug = pd.DataFrame(p_binary_train_aug, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_train_aug

In [ ]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_dev

# Counting

In [ ]:
len(ep_train), len(ep_dev), len(p_train), len(p_dev)

In [ ]:
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

In [ ]:
ep_train = ep_train.drop_duplicates()
p_binary_train_aug = p_binary_train_aug.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

# Export

In [ ]:
%cd /content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11

# train.to_csv('raw_train.csv', index=False)
# dev.to_csv('raw_dev.csv', index=False)
# test.to_csv('raw_test.csv', index=False)

ep_train.to_csv('ce_train.csv', index=False)
p_binary_train_aug.to_csv('pc_binary_train_aug.csv', index=False)
ep_dev.to_csv('ce_dev.csv', index=False)
p_binary_dev.to_csv('pc_binary_dev.csv', index=False)

In [ ]:
# emojis = pd.concat([ep_train.sentence_form, p_train.sentence_form, ep_dev.sentence_form, p_dev.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
# emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))

In [ ]:
df = pd.read_csv('ce_train.csv')
df[df.id == 'nikluge-sa-2022-train-00065']

In [ ]:
df = pd.read_csv('ce_dev.csv')
df

In [ ]:
df = pd.read_csv('pc_binary_train_aug.csv')
df

In [ ]:
df = pd.read_csv('pc_binary_dev.csv')
df